**IMPORTS**

In [86]:
import aux_functions
import importlib

importlib.reload(aux_functions)
from aux_functions import *


import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter
import torch
#from torchvision import datasets, transforms
from torch.utils.data import DataLoader, ConcatDataset
#from sklearn.model_selection import KFold
import numpy as np
import chess
from datetime import datetime

**DATA PROCESSING**

- Importing the pgn data
- Transforming the data to sparce tensors 
- Splitting the data into training and testing

In [87]:
TEST_PERCENT = 0.25

# Load pgn paths
pgns = import_data(5)

# Convert pgns to tensors
board_tensors, next_moves = parse_pgn_to_tensors(pgns)

# Converting the dataset into a custom pytorch one
dataset = ChessDataset(board_tensors, next_moves)

torch.manual_seed(0)
# Splitting the data into train and test
train_dataset, test_data = torch.utils.data.random_split(dataset, [1-TEST_PERCENT, TEST_PERCENT])

print(len(test_data))  # Number of states
print(train_dataset.indices)

845
[414, 235, 1355, 1009, 437, 1064, 2105, 260, 2175, 81, 291, 79, 728, 2653, 2508, 1510, 2518, 2965, 2428, 1621, 2124, 763, 1293, 797, 1454, 1299, 443, 686, 967, 2616, 231, 283, 2045, 49, 2573, 717, 2412, 2931, 1205, 376, 678, 3326, 2258, 986, 1175, 861, 929, 1622, 3254, 3260, 3038, 408, 705, 1465, 949, 1752, 3331, 3113, 222, 365, 879, 128, 3065, 1309, 1241, 3233, 284, 2849, 2422, 2786, 1131, 1511, 92, 1568, 114, 2332, 2917, 1498, 1287, 1329, 126, 2037, 1326, 1383, 995, 1641, 3007, 3150, 293, 671, 2797, 206, 974, 3096, 2249, 229, 2462, 4, 2707, 1108, 35, 1747, 3098, 3055, 442, 1006, 2830, 2053, 393, 2667, 2633, 883, 1056, 1983, 1274, 1399, 3317, 100, 2969, 1744, 3337, 62, 1575, 2861, 1433, 715, 719, 2141, 2795, 2809, 446, 2584, 2959, 2831, 470, 1093, 2034, 381, 2264, 1448, 683, 1388, 1432, 1123, 1047, 941, 2257, 976, 959, 699, 790, 767, 2064, 2994, 2568, 2367, 833, 3168, 1400, 1325, 2240, 277, 2949, 1449, 2513, 3061, 1336, 1401, 3335, 290, 3339, 1367, 2817, 3114, 1415, 787, 2087, 230

**NEURAL NETWORK DESIGN**

In [88]:
BATCH_SIZE = 32

# Whether to do the operations on the cpu or gpu
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class PieceToMoveNet(nn.Module):
    def __init__(self):
        super().__init__()

        # Takes as input a tensor of 14 channels (8x8 board)
        self.conv1 = nn.Conv2d(14, 6, 3)  # 6 filters, 3x3 kernel
        self.pool = nn.MaxPool2d(2, 2)    # Max pooling with 2x2 window
        self.conv2 = nn.Conv2d(6, 16, 3)  # 16 filters, 3x3 kernel

        self.dropout = nn.Dropout(p=0.3)
        self.bn1 = nn.BatchNorm1d(120)
        
        # If starting with 8x8, after two pool layers it becomes 2x2.
        # Output from conv2 will be (16 channels, 1x1 feature maps), flattened to 16 * 1 * 1 = 16
        self.fc1 = nn.Linear(16 * 1 * 1, 120)
        self.fc2 = nn.Linear(120, 84)
        # Predicts the tile to move the piece from (64 possible tiles on the board)
        self.fc3 = nn.Linear(84, 64)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))  # Apply first conv + pooling
        x = F.relu(self.conv2(x)) # Apply second conv to get 16, 1, 1
        x = torch.flatten(x, 1)  # Flatten all dimensions except batch size
        x = F.relu(self.bn1(self.fc1(x)))  # Fully connected layer 1
        x = self.dropout(x)
        x = F.relu(self.fc2(x))  # Fully connected layer 2
        x = self.fc3(x)          # Output layer (no activation, logits for classification)
        return x

piece_to_move_net = PieceToMoveNet()
# Move the network to gpu / cpu befor initializing the optimizer
piece_to_move_net.to(device)

optimizer = optim.Adam(piece_to_move_net.parameters(), lr=1e-4, weight_decay=1e-5)
loss_fn = torch.nn.CrossEntropyLoss()

**TRAINING LOOP**

In [90]:
EPOCHS = 50
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
writer = SummaryWriter(f"runs/piece_to_move_{timestamp}")

torch.manual_seed(1)
train_data, validation_data = torch.utils.data.random_split(train_dataset, [1-TEST_PERCENT, TEST_PERCENT])

training_loader = torch.utils.data.DataLoader(train_data, BATCH_SIZE, shuffle=True, pin_memory=True)
validation_loader = torch.utils.data.DataLoader(validation_data, BATCH_SIZE, shuffle=True, pin_memory=True)

def train_one_epoch(epoch_index: int, tb_writer, optimizer, training_loader, loss_fn): 
    running_loss = 0.
    last_loss = 0.

    for i, data in enumerate(training_loader):
        inputs = data[0]
        # Extracting only the tile of the piece to move
        labels = data[1][:, 0]

        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = piece_to_move_net(inputs)

        loss = loss_fn(outputs, labels)

        loss.backward()

        optimizer.step()

        running_loss += loss.item()
        if i % 1000 == 999: 
            last_loss = running_loss / 1000
            print(f" batch {i + 1}, loss: {last_loss}")
            tb_x = epoch_index * len(training_loader) + i + 1
            tb_writer.add_scalar("Loss/train", last_loss, tb_x)
            running_loss = 0.
    
    if last_loss == 0.:
        last_loss = running_loss / (i+1)

    return last_loss


def train_multiple_epochs(n_epochs, model, writer, validation_loader, loss_fn):
    best_vloss = 1_000_000

    for epoch in range(EPOCHS): 
        print(f"EPOCH {epoch + 1}: ")

        model.train(True)
        avg_loss = train_one_epoch(epoch, writer, optimizer, training_loader, loss_fn)
        running_vloss = 0.0

        model.eval()

        total_vbatches = 0
        with torch.no_grad():
            for i, v_data in enumerate(validation_loader):
                
                vinputs = v_data[0]
                vlabels = v_data[1][:,0]

                vinputs = vinputs.to(device)
                vlabels = vlabels.to(device)

                voutputs = model(vinputs)
                vloss = loss_fn(voutputs, vlabels)
                running_vloss += vloss.item()

        avg_vloss = running_vloss / (i+1)
        print(f"LOSS train {avg_loss}, valid {avg_vloss}")

        writer.add_scalars("Training vs Validation Loss", {
            "Training": avg_loss, "Validation": avg_vloss}, 
            epoch + 1)
        writer.flush()
        
        if avg_vloss < best_vloss:
            best_vloss = avg_loss
            model_path = f"model_{timestamp}_{epoch}"
            torch.save(model.state_dict(), model_path)

        epoch += 1


train_multiple_epochs(EPOCHS, piece_to_move_net, writer, validation_loader, loss_fn)



[1752, 724, 356, 2155, 2398, 630, 1726, 1768, 1510, 1001, 1351, 1778, 928, 1844, 2139, 2161, 595, 2498, 2517, 1209, 1568, 582, 213, 1155, 670, 2229, 1365, 789, 1919, 1287, 21, 1037, 554, 368, 1058, 122, 1253, 703, 2401, 515, 1228, 466, 797, 1930, 1457, 1026, 1326, 836, 338, 345, 842, 1206, 43, 221, 2493, 742, 2167, 2131, 652, 1637, 1127, 2156, 146, 1415, 1693, 890, 2427, 2524, 2469, 1409, 26, 2278, 1667, 1475, 438, 1073, 2365, 1136, 2092, 2455, 137, 1937, 2504, 1586, 1269, 611, 2376, 2458, 168, 17, 1103, 2077, 1809, 1248, 1295, 599, 2520, 134, 964, 1898, 869, 1642, 2470, 1271, 1182, 1417, 942, 2227, 1526, 1852, 1602, 756, 1282, 944, 1309, 1051, 1571, 707, 1226, 1679, 1370, 416, 1756, 1246, 1527, 1666, 2509, 151, 651, 2134, 2441, 2461, 455, 994, 1575, 1833, 418, 1604, 1765, 1822, 383, 2110, 564, 184, 2095, 2348, 1823, 1961, 761, 1284, 1217, 1862, 1364, 1874, 193, 1313, 1470, 2253, 1503, 1521, 494, 1781, 1016, 522, 931, 1489, 1926, 427, 2269, 1353, 2232, 1000, 1694, 781, 1339, 2067, 1868

In [ ]:
def generate_mask(board: chess.Board, outputs: np.arry, labels: np.array) -> np.array:
    """Creates mask with legal moves from the current board state"""

    mask = np.zeros((8,8)) # 8x8 mask for the chessboard

    # Obtaining legal moves from the board
    legal_moves = list(board.legal_moves)

    # Indicating with 1s the valid squares
    for move in legal_moves:
        to_square = move.to_square
        to_row, to_col = divmod(to_square, 8)
        mask[to_row, to_col] = 1 # A valid square

    # Reshaping mask to match output and labels
    move_mask = mask.flatten() # Converts 8*8 2D array to a 1D array with 64 elements

    masked_outputs = outputs * move_mask
    masked_labels = labels * move_mask

    return masked_outputs, masked_labels

def update_board( board: chess.Board, move: chess.Move ) -> chess.Board:
    """This function is responsible for updating the board everytime a move is made"""

    board.push(move) # Add move to the board
    
    return move

**CROSS VALIDATION**


In [ ]:
def reset_weights(model):
    """Resets the weights of the model, so the model is trained with randomly initalized weights"""

    # List of layers containing reset parameters
    layer_types = [nn.Conv2d, nn.Linear, nn.BatchNorm2d]

    # Iterating through all layers of the model
    for layer in model.modules():
        # Check layers with reset parameters
        if type(layer) in layer_types:
            layer.reset_parameters()

    